# Download Sentinel-2 NIR band images from AWS and crop

Requires AWS and GDAL command line functionality.

In [91]:
import os
import subprocess
import numpy as np
import shutil
import geopandas as gpd
import fiona
import shapely
# Enable fiona KML file reading driver
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

In [109]:
# set basepath and outputpath (same)
basepath = '/Volumes/SURGE_DISK/S2_LO/'
outputpath = basepath 

# 0) Automatically the identify S2 tile that overlaps your AOI

Must download the S2 tiling grid from here: https://sentinel.esa.int/web/sentinel/missions/sentinel-2/data-products

In [110]:
AOIpath = '/Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_WGS.shp'
S2grid_path = '/Users/jukesliu/Documents/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml'

In [111]:
aoi_gdf = gpd.read_file(AOIpath)
aoi = aoi_gdf.geometry.values
aoi

<GeometryArray>
Length: 1, dtype: geometry

In [115]:
# open the S2 tile grid
s2grid_shp = fiona.open(S2grid_path)

# find the overlapping tile
tilename = 'None'
for feature in s2grid_shp:
    tile = shapely.geometry.Polygon(feature['geometry']['geometries'][0]['coordinates'][0])
#     print(feature['properties']['Name'])
    if tile.overlaps(aoi[0]):
        tilename=feature['properties']['Name']
        print(tilename)
#         break # stop searching
print(tilename)

07VFG
07VFH
08VLM
08VLN
08VLN


In [113]:
# construct the folderpath
tilefolder = str(int(tilename[0:2]))+'/'+tilename[2:3]+'/'+tilename[3:]+'/'
print(tilefolder)

ValueError: invalid literal for int() with base 10: 'No'

## Or set it manually here:

In [122]:
tilefolder = '8/V/LM/'

## Explore files available on AWS manually:

In [121]:
# explore files manually:
!export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; aws s3 ls s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/ --no-sign-request

                           PRE 2017/
                           PRE 2018/
                           PRE 2019/
                           PRE 2020/
                           PRE 2021/
                           PRE 2022/
                           PRE 2023/


# 1) Set dates and download

Syntax:

aws --no-sign-request s3 cp s3://landsat-pds/c1/L8/031/005/ Output/path/LS8aws/Path031_Row005/ --recursive --exclude "*" --include "*MTL.txt"

In [123]:
# SET YEARS AND MONTHS TO DOWNLOAD
years = [2020,2021]; print(years) # set year(s) to download
months = np.linspace(1,12,12); print(months) # set month(s) to download

[2020, 2021]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [124]:
# loop through and download
for year in years:
    for month in months:
        year = str(year); month = str(int(month)) # convert to strings
        print('Downloading', year, month)

        cmd = 'export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; '
        cmd += 'aws --no-sign-request s3 cp s3://sentinel-cogs/sentinel-s2-l2a-cogs/'+tilefolder+year+'/'+month+'/'
        cmd += ' '+outputpath+' --recursive --exclude "*/*" --include "*/B08.tif"'
        print(cmd)
        
        subprocess.run(cmd, check=True, shell=True)

export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; aws --no-sign-request s3 cp s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/1/ /Volumes/SURGE_DISK/S2_LO/ --recursive --exclude "*/*" --include "*/B08.tif"
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/1/S2A_8VLM_20200115_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200115_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/1/S2A_8VLM_20200118_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200118_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/1/S2A_8VLM_20200112_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200112_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/1/S2A_8VLM_20200112_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200112_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/1/S2A_8VLM_2

download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/2/S2B_8VLM_20200206_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200206_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/2/S2B_8VLM_20200209_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200209_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/2/S2B_8VLM_20200216_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200216_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/2/S2B_8VLM_20200212_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200212_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/2/S2B_8VLM_20200216_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200216_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/2/S2B_8VLM_20200212_0_L2A/B08.tif to ../../../../../.

download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/3/S2B_8VLM_20200317_3_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200317_3_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/3/S2B_8VLM_20200320_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200320_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/3/S2B_8VLM_20200317_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200317_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/3/S2B_8VLM_20200323_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200323_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/3/S2B_8VLM_20200320_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200320_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/3/S2B_8VLM_20200327_0_L2A/B08.tif to ../../../../../.

download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/5/S2A_8VLM_20200504_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200504_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/5/S2A_8VLM_20200501_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200501_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/5/S2A_8VLM_20200501_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200501_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/5/S2A_8VLM_20200507_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200507_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/5/S2A_8VLM_20200504_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200504_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/5/S2A_8VLM_20200507_1_L2A/B08.tif to ../../../../../.

download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/6/S2A_8VLM_20200620_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200620_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/6/S2A_8VLM_20200616_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200616_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/6/S2A_8VLM_20200620_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200620_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/6/S2A_8VLM_20200623_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200623_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/6/S2A_8VLM_20200623_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200623_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2020/6/S2A_8VLM_20200626_0_L2A/B08.tif to ../../../../../.

...

download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/7/S2A_8VLM_20210705_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210705_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/7/S2A_8VLM_20210705_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210705_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/7/S2A_8VLM_20210708_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210708_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/7/S2A_8VLM_20210701_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210701_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/7/S2A_8VLM_20210701_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210701_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/7/S2A_8VLM_20210708_1_L2A/B08.tif to ../../../../../.

download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/8/S2A_8VLM_20210814_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210814_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/8/S2A_8VLM_20210810_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210810_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/8/S2A_8VLM_20210814_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210814_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/8/S2A_8VLM_20210817_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210817_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/8/S2A_8VLM_20210817_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210817_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/8/S2A_8VLM_20210824_0_L2A/B08.tif to ../../../../../.

download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/9/S2A_8VLM_20210916_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210916_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/9/S2A_8VLM_20210919_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210919_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/9/S2A_8VLM_20210919_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210919_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/9/S2A_8VLM_20210923_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210923_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/9/S2A_8VLM_20210926_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210926_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/9/S2A_8VLM_20210923_1_L2A/B08.tif to ../../../../../.

download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/10/S2A_8VLM_20211029_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211029_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/10/S2A_8VLM_20211029_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211029_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/10/S2B_8VLM_20211001_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211001_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/10/S2B_8VLM_20211001_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211001_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/10/S2B_8VLM_20211008_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211008_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/10/S2B_8VLM_20211004_0_L2A/B08.tif to ../../../.

download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/11/S2B_8VLM_20211117_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211117_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/11/S2B_8VLM_20211113_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211113_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/11/S2B_8VLM_20211117_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211117_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/11/S2B_8VLM_20211120_0_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211120_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/11/S2B_8VLM_20211120_1_L2A/B08.tif to ../../../../../../../Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211120_1_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/8/V/LM/2021/11/S2B_8VLM_20211127_0_L2A/B08.tif to ../../../.

# 2) Rename the files using acquisition and consolidate from subfolders:

In [125]:
for img in os.listdir(basepath):
    if len(img) == 23 or len(img) == 24:
        files = os.listdir(basepath+img)
        for file in files:
            if file == 'B08.tif':
#                 print(img)
                spath = basepath+img+'/B08.tif'
                tpath = basepath+img[:18]+'_B08.tif'
                os.rename(spath, tpath)
                shutil.rmtree(basepath+img)
                
#                 print(file)

# 3) Crop images to AOI:

In [126]:
# set paths
imagepath = '/Volumes/SURGE_DISK/S2_LO/' # where images are
boxpath = '/Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp' #  AOI box (UTM)

In [127]:
for image in os.listdir(imagepath):
    if not os.path.exists(imagepath+image[:-4]+'_clipped.tif') and image.endswith('B08.tif'):
        crop_cmd = 'gdalwarp -cutline '+boxpath+' -crop_to_cutline '+imagepath+image+" "+imagepath+image[:-4]+'_clipped.tif'
        print(crop_cmd)
        subprocess.call(crop_cmd, shell=True)

gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200115__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200115__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200115__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200115__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200115__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200115__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200125__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200125__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200125__B08.tif [1/1] : 0Usin

Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200123__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200123__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200123__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200123__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200130__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200130__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200130__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200130__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200130__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200130__B08_clipped.tif.


...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200206__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200206__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200206__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200206__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200206__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20200206__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200308__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200308__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200318__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200318__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200318__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200318__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200322__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200322__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200322__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200322__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200322__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200322__B08_clipped.tif.


...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200401__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200401__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200401__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200401__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200401__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200401__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200411__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20200411__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

...

Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210906__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210906__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210906__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210906__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210903__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210903__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210903__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210903__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20210903__B08.tif to destination /Volumes/SURGE_DIS

...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20210911__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20210911__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20210911__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20210911__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20210911__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20210911__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211009__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211009__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211023__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211023__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211023__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211023__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211029__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211029__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211029__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211029__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211029__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211029__B08_clipped.tif.


...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211102__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211102__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211102__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211102__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211102__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211102__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211112__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20211112__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211127__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211127__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211127__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211127__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211123__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211123__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211123__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211123__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211123__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20211123__B08_clipped.tif.
